In [8]:
import os
import gc
from glob import glob
import warnings
import json
import math

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.colors as colors

import numpy as np
import pandas as pd

import cv2 as cv
import scipy
from skimage import measure, draw, filters, morphology

warnings.simplefilter("ignore")

In [9]:
#setup paths for inputs
image_folder = '/home/meso/data/cross-section-photos/analysis/photos'
reference_csv_ffn = '/home/meso/data/cross-section-photos/analysis/photos/xsec_reference_measurements.csv'
centres_json_ffn = '/home/meso/data/cross-section-photos/analysis/photos/melb_20200119_hail.json'

#setup paths for outputs
out_root = '/home/meso/data/cross-section-photos/analysis/pipeline_img'
stats_root = '/home/meso/data/cross-section-photos/analysis/pipeline_stats'
resized_eq_root = '/home/meso/data/cross-section-photos/analysis/resized_eq_img'

In [81]:
def area_by_shoelace(x, y):
    """
    Calculate area within polygon defined by x,y points
    Assumes x,y points go around the polygon in one direction
    """
    return abs( sum(i * j for i, j in zip(x,             y[1:] + y[:1]))
               -sum(i * j for i, j in zip(x[1:] + x[:1], y            ))) / 2

def moving_average(a, n=3):
    """
    return the moving average of a 1D vector.
    total length is preserved by using same values on edges
    """
    return np.convolve(a, np.ones((n,))/n, mode='same')

def find_peak_width(lum_data, peaks):
    
    #config
    fall_perc = 0.30 #luminosity percentage
    merge_dist=15 #range pixels
    edge_dist=20 #range pixels
    min_width=10 #range pixels
    
    
    left_pass_1 = []
    right_pass_1 = []
    
    #find edge
    edge_idx = np.where(lum_data>0)[0][-1]
    
    for peak_idx in peaks:
        #extract peak lumosity
        peak_lum = lum_data[peak_idx]
        #extract threshold for lumosity
        peak_lower_limit = peak_lum - (peak_lum*fall_perc)
        #setup arrays for detecting edges of dry growth region
        lum_data_left = lum_data.copy()
        lum_data_left[peak_idx:] = peak_lum #replace lum values on the right side of the peak with peak_lum, preserving the left side
        lum_data_right = lum_data.copy()
        lum_data_right[:peak_idx] = peak_lum #replace lum values on the left side of the peak with peak_lum, preserving the right side
        #find sides of peak
        #right
        try:
            peak_right = np.where(lum_data_right<peak_lower_limit)[0][0] #use first value below the threshold to find the right point. Will always work because of background
        except:
            #take the argmin if this fails
            peak_right = np.argmin(lum_data_right)
        #enforce edge limit on peak right
        if peak_right > edge_idx:
            peak_right = edge_idx
        #left
        try:
            peak_left = np.where(lum_data_left<peak_lower_limit)[0][-1] #use last value below threshold to find left point
        except:
            #cases for close peaks to centre, and transect does not reduce below threshold.
            peak_left = peak_idx - (peak_right-peak_idx) #reflection of right peak
        #enforce 0
        if peak_left < 0:
            peak_left = 0        

        #append
        left_pass_1.append(peak_left)
        right_pass_1.append(peak_right)

    left_pass_1  = np.array(left_pass_1)
    right_pass_1 = np.array(right_pass_1)
    peak_pass_1 = np.array(peaks)
        
    #merge overlapping peaks
    left_pass_2 = []
    right_pass_2 = []
    peak_pass_2 = []
    proc_flag = np.zeros_like(left_pass_1, dtype=bool)
    for i, peak in enumerate(peak_pass_1):
        #check if already processes
        if proc_flag[i]:
            continue
        #check if layer extends from the centre (erase and skip)
        if left_pass_1[i] == 0:
            #write -999 which removed this peak in pass 3 (using the width)
            left_pass_1[i] = -999
            right_pass_1[i] = -999
            continue
        #find if edges overlap with multiple left/right regions
        current_left = left_pass_1[i]
        current_right = right_pass_1[i]
        overlap = np.where(np.logical_and(current_left<=right_pass_1, current_right>=left_pass_1))[0]
        #overlap = np.where(np.logical_and(left_pass_1<peak, right_pass_1>peak))[0]
        #find min of those points
        left_pass_2.append(np.min(left_pass_1[overlap]))
        right_pass_2.append(np.max(right_pass_1[overlap]))
        overlap_peak_lum = lum_data[peak_pass_1[overlap]] #find luminosity values of overlapping peaks
        peak_pass_2.append(peak_pass_1[overlap[np.argmax(overlap_peak_lum)]]) #only append the index of the highest luminosity value
        #assign as processed
        proc_flag[overlap] = True
    
    #filtering to remove very narrow peaks and peaks centred very close to the edge (using a 20pix limit)
    left_pass_3 = []
    right_pass_3 = []
    peak_pass_3 = []
    for i, peak in enumerate(peak_pass_2):
        width = right_pass_2[i] - left_pass_2[i]
        #check size of peak width and distance from edge
        if (edge_idx - peak) > edge_dist and width > min_width:
            left_pass_3.append(left_pass_2[i])
            right_pass_3.append(right_pass_2[i])
            peak_pass_3.append(peak_pass_2[i])

    #output
    left_array = np.array(left_pass_3)
    right_array = np.array(right_pass_3)
    peak_array = np.array(peak_pass_3)
        
    return left_array, right_array, peak_array, edge_idx

In [123]:
def worker(image_ffn, c_coord, ref_coord, emb_coord, img_ref_mm):
    
    #config
    gaussian_filter_sigma = 4
    azimuth_spacing = 10 #degrees
    ray_prominence = 30 #luminosity
    ray_distance = 30 #range dim steps
    ray_height = 80 #luminosity
    min_azi_with = 4 #azi dim steps
    layer_prominence = 5 #dry growth pixels
    layer_distance = 30 #range dim steps
    
    img_fn = os.path.basename(image_ffn)
    img_id = img_fn[0:2]
    print('processing', image_ffn)
    #unpack inputs
    cx = c_coord[0]
    cy = c_coord[1]
    
    #work out pixel scaling
    ref_coord_x = ref_coord[0]
    ref_coord_y = ref_coord[1]
    ref_len = np.sqrt((ref_coord_x[0]-ref_coord_x[1])**2 + (ref_coord_y[0]-ref_coord_y[1])**2)
    ref_pix_mm = ref_len/img_ref_mm
    #work out embryo sa
    if emb_coord is None:
        emb_sa = None
    else:
        emb_coord_x = emb_coord[0] + [emb_coord[0][0]]
        emb_coord_y = emb_coord[1] + [emb_coord[1][0]]
        emb_sa = area_by_shoelace(emb_coord_x, emb_coord_y)

    #transform into hsv colorspace
    img_data = cv.imread(image_ffn)
    img_data_hls = cv.cvtColor(img_data, cv.COLOR_BGR2HLS)
    lum_img = img_data_hls[:,:,1]
    img_size = np.shape(lum_img)
    lum_img_smooth = filters.gaussian(lum_img, sigma=gaussian_filter_sigma)*255
    #define and plot transect lines
    pixel_range   = np.max(img_size) #pixels (max possible distance)
    #init azimuth list
    azimuth_dim = np.arange(0,360,azimuth_spacing)
    range_dim = np.arange(0,pixel_range)
    #init range element size
    range_bin = range_dim[1] - range_dim[0]
    azi_bin = azimuth_dim[1] - azimuth_dim[0]
    size_dim = np.zeros_like(range_dim)
    for i, bin_range in enumerate(range_dim):
        size_dim[i] = (((bin_range+(range_bin/2))**2)*math.pi - ((bin_range-(range_bin/2))**2)*math.pi)/360*azi_bin
    
    #init azimuth_range array
    layer_array = np.zeros((pixel_range, len(azimuth_dim))) -1 #set all to be outside
    
    line_x_list = []
    line_y_list = []
    line_lum_list = []
    peaks_idx_list = []
    peaks_x_list = []
    peaks_y_list = []
    peaks_azi_list = []
    edge_idx_list = []
    cl_x_list = []
    cl_y_list = []
    cr_x_list = []
    cr_y_list = []
    c_width_list = []
    
    #for each azimuth
    for i, azi in enumerate(azimuth_dim):
        # create indicies for lines from centroid along search azimuth rays
        end_x    = int(cx + pixel_range*math.cos(math.radians(azi))) #2x ensures it already reaches the edges
        end_y    = int(cy + pixel_range*math.sin(math.radians(azi)))
        #generate line points
        line_x   = np.linspace(cx, end_x, pixel_range).astype(np.int)
        line_y   = np.linspace(cy, end_y, pixel_range).astype(np.int)
        #enforce boundaries
        mask_x = np.logical_and(line_x >= 0, line_x <= img_size[1]-1)
        mask_y = np.logical_and(line_y >= 0, line_y <= img_size[0]-1)
        mask = np.logical_and(mask_x, mask_y)
        line_x = line_x[mask]
        line_y = line_y[mask]
        
        # extract data along
        line_lum = lum_img_smooth[line_y, line_x].astype(int)
        #measure peaks
        peaks_idx, _ = scipy.signal.find_peaks(line_lum, prominence=ray_prominence, distance=ray_distance, height=ray_height)

        #find width of peaks and combine overlapping peaks
        left_idx, right_idx, new_peak_idx, edge_idx = find_peak_width(line_lum, peaks_idx)        
        
        #add to layer_array
        layer_vec = np.zeros(pixel_range,)
        for j,_ in enumerate(new_peak_idx):
            layer_vec[left_idx[j]:right_idx[j]+1] = 1
        layer_vec[edge_idx:] = -1
        layer_array[:,i] = layer_vec

        #check for valid peaks
        if len(new_peak_idx) == 0:
            continue
        
        #store
        line_x_list.append(line_x)
        line_y_list.append(line_y)
        line_lum_list.append(line_lum)
        peaks_idx_list.append(new_peak_idx)
        peaks_x_list.append(line_x[new_peak_idx])
        peaks_y_list.append(line_y[new_peak_idx])
        peaks_azi_list.append(np.zeros_like(new_peak_idx)+azi)
        cl_x_list.append(line_x[left_idx])
        cl_y_list.append(line_y[left_idx])
        cr_x_list.append(line_x[right_idx])
        cr_y_list.append(line_y[right_idx])
        edge_idx_list.append(edge_idx)
    
    #flatten centre info arrays
    cl_x_array = np.concatenate(cl_x_list).ravel()
    cl_y_array = np.concatenate(cl_y_list).ravel()
    cr_x_array = np.concatenate(cr_x_list).ravel()
    cr_y_array = np.concatenate(cr_y_list).ravel()
    peak_idx_array = np.concatenate(peaks_idx_list).ravel()
    peak_x_array = np.concatenate(peaks_x_list).ravel()
    peak_y_array = np.concatenate(peaks_y_list).ravel()
    peaks_azi_array = np.concatenate(peaks_azi_list).ravel()
    
    #remove small regions from layer_array by checking the number of unique azimuthal gates (threshold set at 3)
    layer_label = morphology.label(layer_array==1)
    label_max = np.max(layer_label)
    azimuth_grid, _ = np.meshgrid(azimuth_dim, range_dim)
    for label in range(1,label_max+1):
        unqiue_azi_set = np.unique(azimuth_grid[layer_label==label])
        azi_width = len(unqiue_azi_set)
        if azi_width <= min_azi_with:
            layer_array[layer_label==label] = 2
        
    #layer_valid = morphology.remove_small_objects(layer_array==1, min_size=100)
    #layer_array[layer_valid]=2
    layer_valid = layer_array==1
    
    #compute range density
    range_density = np.sum(layer_valid, axis=1)
    #smooth
    range_density_smooth = moving_average(range_density, 10)
    #find peaks in range density
    layer_peaks_idx, _ = scipy.signal.find_peaks(range_density_smooth, prominence=layer_prominence, distance=layer_distance)
    #for each peak, centre of mass and total count
    layer_lower_min_idx = []
    layer_upper_min_idx = []
    layer_sa_list = []
    layer_weighted_centre_list = []
    for i, peak in enumerate(layer_peaks_idx):
        #find index of upper and lower minimum
        if i==0:
            lower_min = np.where(range_density_smooth>0)[0][0] #first nonzero value before the first peak
        else:
            lower_min = layer_peaks_idx[i-1] + np.argmin(range_density_smooth[layer_peaks_idx[i-1]:peak])
            
        if i+1==len(layer_peaks_idx):
            upper_min = peak + np.where(range_density_smooth[peak:]==0)[0][0] #first zero value after the final peak
        else:
            upper_min = peak + np.argmin(range_density_smooth[peak:layer_peaks_idx[i+1]])
        layer_lower_min_idx.append(lower_min)
        layer_upper_min_idx.append(upper_min)
        #calculate count
        layer_sa = range_density[lower_min:upper_min+1]*size_dim[lower_min:upper_min+1]
        layer_sa_list.append(np.sum(layer_sa))
        #calculate centre of mass radius
        radius_sum = np.sum(layer_sa*range_dim[lower_min:upper_min+1])
        layer_weighted_centre_list.append(radius_sum/np.sum(layer_sa))
        
    fig = plt.figure(facecolor='white',figsize=[12, 11])
    fig.suptitle(f"Hailstone {img_id}", fontsize=16)
    
    #plot azimuth data on image
    plt.subplot(221)
    plt.gca().invert_yaxis()
    plt.imshow(lum_img, cmap='gray')
    plt.plot(cx, cy, 'r*', markersize=16, label='Embryo Centre')
    try:
        plt.plot(emb_coord_x, emb_coord_y, 'r-', linewidth=3, label='Embryo Edge')
    except:
        #no dry embryo
        pass
    plt.plot(ref_coord_x, ref_coord_y, 'r--', linewidth=3, label='Reference')
    for i, _ in enumerate(cl_x_array):
        if i == 0:
            #mark peaks
            plt.plot(peak_x_array[i], peak_y_array[i], 'b.', markersize=8, label='Peak')
            #mark edges
            plt.plot([cl_x_array[i],cr_x_array[i]],[cl_y_array[i],cr_y_array[i]], 'b-', label='Peak Width')
        else: #plot without labels
            #mark peaks
            plt.plot(peak_x_array[i], peak_y_array[i], 'b.', markersize=8)
            #mark edges
            plt.plot([cl_x_array[i],cr_x_array[i]],[cl_y_array[i],cr_y_array[i]], 'b-')
    plt.legend(loc='lower left')
    #plt.axis('equal')
    plt.axis('off')
    plt.title('(a) Radial Transect Detection', fontsize=14)
    
    
    
    #plot azimuth/range arrat on image
    hail_colors = [
    "black",
    "dimgrey",
    "white",
    "yellow"]
    hail_cmap = colors.ListedColormap(hail_colors)   
    
    plt.subplot(222)
    cmap = matplotlib.cm.get_cmap('viridis', 4)
    img = plt.pcolor(azimuth_dim, range_dim, layer_array, vmin=-1.5, vmax=2.5, cmap=hail_cmap)
    cbar = plt.colorbar(img, ticks=[-1, 0, 1, 2], orientation='horizontal')
    cbar.ax.set_xticklabels(['Background', 'Wet Growth', 'Dry Growth', 'Filtered Dry Growth'])
    plt.axis('auto')
    plt.ylim([0,700])
    plt.xlabel('Azimuth (deg)')
    plt.ylabel('Range (pix.)')
    plt.title('(b) Radial Transect Analysis', fontsize=14)
    
    
    #plot range density
    plt.subplot(223)  
    plt.plot(range_dim, range_density_smooth, 'k--')
    plt.plot(range_dim[layer_peaks_idx], range_density_smooth[layer_peaks_idx], marker="o", markerfacecolor='b',
             linewidth=0, label='Peak', markersize=8)
    plt.plot(layer_weighted_centre_list, range_density_smooth[np.array(layer_weighted_centre_list).astype('int')], marker="o", markerfacecolor='k',
             linewidth=0, label='Area Weighted Centre', markersize=8)
    plt.plot(range_dim[layer_lower_min_idx], range_density_smooth[layer_lower_min_idx], marker=7, markerfacecolor='b', linewidth=0, label='Layer Edge', markersize=8)
    plt.plot(range_dim[layer_upper_min_idx], range_density_smooth[layer_upper_min_idx], marker=7, markerfacecolor='b', linewidth=0, markersize=8)
    plt.xlim([-10,700])
    plt.ylim([-1,np.max(range_density_smooth)+5])
    plt.xlabel('Range (pix.)')
    plt.ylabel('Dry Growth Count (pix.)')
    plt.title('(c) Layer Detection', fontsize=14)
    plt.legend()
    
    #draw equivalent circular hail
    ax = plt.subplot(224)
    ax.set_xlim((-35,35))
    ax.set_ylim((-35,35))
    ax.set_xlabel('(mm)')
    ax.set_ylabel('(mm)')
    #ax.set_aspect('equal')
    ax.set_facecolor('black')
    #draw entire hail
    mean_edge_radius = np.mean(edge_idx_list)/ref_pix_mm
    max_edge_radius = np.max(edge_idx_list)/ref_pix_mm
    n_layers = len(layer_sa_list)
    background = plt.Circle((0, 0), mean_edge_radius, color='dimgrey')
    ax.add_artist(background)
    total_dry_sa = 0
    for i in range(len(layer_sa_list)-1, -1, -1): #loop backwards
        r = layer_weighted_centre_list[i]
        sa = layer_sa_list[i]
        total_dry_sa += sa/(ref_pix_mm**2)
        r_inner = math.sqrt(r**2-(sa/(2*math.pi)))/ref_pix_mm
        r_outer = math.sqrt((sa/(2*math.pi))+r**2)/ref_pix_mm
        circle_inner = plt.Circle((0, 0), r_inner, color='dimgrey')
        circle_outer = plt.Circle((0, 0), r_outer, color='white')
        ax.add_artist(circle_outer)
        ax.add_artist(circle_inner)
    #draw embryo
    if emb_sa is not None:
        r_embryo = np.sqrt(emb_sa/math.pi)/ref_pix_mm
        circle_embryo = plt.Circle((0, 0), r_embryo, color='white')
        ax.add_artist(circle_embryo)
    plt.title('(d) Equivalent Cross-Section', fontsize=14)
    
    
    
#     #draw scaled hail image in greyscale
#     ax = plt.subplot(224)
#     ax.set_xlim((-35,35))
#     ax.set_ylim((-35,35))
#     ax.set_xlabel('(mm)')
#     ax.set_ylabel('(mm)')
#     ax.set_aspect('equal')
#     ax.set_facecolor('k')
#     x = (np.arange(0,img_size[1])-int(img_size[1]/2))/ref_pix_mm
#     y = (np.arange(0,img_size[0])-int(img_size[0]/2))/ref_pix_mm
#     plt.pcolor(x, y, np.flipud(lum_img), cmap='gray')
#     plt.title('scaled cross section')
    
    plt.tight_layout()
    plt.savefig(f'{out_root}/analysis_{os.path.basename(image_ffn)}')
    plt.clf()
    plt.close('all')
    
    
    #save some stats
    np.savez(f'{stats_root}/stats_{img_id}.npz', mean_edge_radius=mean_edge_radius,
            max_edge_radius=max_edge_radius, total_dry_sa=total_dry_sa, n_layers=n_layers)
    

    
#     #draw equivalent circular hail in its own figure
#     fig = plt.figure(figsize=[10,10])
#     fig.set_facecolor('k')
#     ax = plt.subplot(111)
#     ax.set_xlim((-30,30))
#     ax.set_ylim((-30,30))
#     ax.set_aspect('equal')
#     ax.set_facecolor('k')
#     #draw entire hail
#     mean_edge_radius = np.mean(edge_idx_list)/ref_pix_mm
#     background = plt.Circle((0, 0), mean_edge_radius, color=[0.5,0.5,0.5])
#     ax.add_artist(background)
#     for i in range(len(layer_sa_list)-1, -1, -1): #loop backwards
#         r = layer_weighted_centre_list[i]
#         sa = layer_sa_list[i]
#         r_inner = math.sqrt(r**2-(sa/(2*math.pi)))/ref_pix_mm
#         r_outer = math.sqrt((sa/(2*math.pi))+r**2)/ref_pix_mm
#         circle_inner = plt.Circle((0, 0), r_inner, color=[0.5,0.5,0.5])
#         circle_outer = plt.Circle((0, 0), r_outer, color='w')
#         ax.add_artist(circle_outer)
#         ax.add_artist(circle_inner)
#     #draw embryo
#     if emb_sa is not None:
#         r_embryo = np.sqrt(emb_sa/math.pi)/ref_pix_mm
#         circle_embryo = plt.Circle((0, 0), r_embryo, color='w')
#         ax.add_artist(circle_embryo)
#     #draw grids
#     x = (np.arange(0,img_size[1])-int(img_size[1]/2))/ref_pix_mm
#     y = (np.arange(0,img_size[0])-int(img_size[0]/2))/ref_pix_mm
#     grid_coord = [-25,-15,-5,5,15,25]
#     for i in grid_coord:
#         plt.plot([i,i],[-30,30],'w:', lw=0.5)
#         plt.plot([-30,30],[i,i],'w:', lw=0.5)
#     ax.set_axis_off()
#     plt.savefig(f'{resized_eq_root}/resized_eq_{os.path.basename(image_ffn)}', bbox_inches='tight', pad_inches=0)
#     plt.clf()
#     plt.close('all')
    
    gc.collect()
    

In [124]:
#build arguments

#read reference sizes
df = pd.read_csv(reference_csv_ffn)

#build image file and centroid arguments from json
image_args_list = []
with open(centres_json_ffn) as json_fh:  
    data = json.load(json_fh)
    for item in data.keys():
        #extract image name and id
        img_fn = data[item]['filename']
        img_id = int(img_fn[0:2])
        #init objects
        c_coord = None
        ref_coord = None
        emb_coord = None
        #extract shape objects
        for shape in data[item]['regions']:
            shape = shape['shape_attributes']
            if shape['name'] == 'point':
                c_coord = [shape['cx'], shape['cy']]
            if shape['name'] == 'polygon':
                emb_coord = [shape['all_points_x'], shape['all_points_y']]
            if shape['name'] == 'polyline':
                ref_coord = [shape['all_points_x'], shape['all_points_y']]    
        if emb_coord == None:
            print('no dry embryo for', img_id)
        #check for missing metadata
        if c_coord == None or ref_coord == None:
            print('error: missing json data for if', img_id)
            continue
        
        #extract reference measurement
        try:
            img_ref_mm = float(df.size_mm[df.id==img_id])
        except:
            print('error: failed to find reference measurement to match id', img_id)
            continue
        #build args
        image_args_list.append((f'{image_folder}/{img_fn}', c_coord, ref_coord, emb_coord, img_ref_mm))

no dry embryo for 2
no dry embryo for 5
no dry embryo for 9
no dry embryo for 24
no dry embryo for 25
no dry embryo for 26
no dry embryo for 27
no dry embryo for 29
no dry embryo for 34


In [ ]:
for item in image_args_list:
    worker(*item)


processing /home/meso/data/cross-section-photos/analysis/photos/01.png
processing /home/meso/data/cross-section-photos/analysis/photos/02.png
processing /home/meso/data/cross-section-photos/analysis/photos/03.png
processing /home/meso/data/cross-section-photos/analysis/photos/04.png
processing /home/meso/data/cross-section-photos/analysis/photos/05.png
processing /home/meso/data/cross-section-photos/analysis/photos/06.png
processing /home/meso/data/cross-section-photos/analysis/photos/07.png
processing /home/meso/data/cross-section-photos/analysis/photos/08.png
processing /home/meso/data/cross-section-photos/analysis/photos/09.png
processing /home/meso/data/cross-section-photos/analysis/photos/10.png
processing /home/meso/data/cross-section-photos/analysis/photos/11.png
processing /home/meso/data/cross-section-photos/analysis/photos/12.png
processing /home/meso/data/cross-section-photos/analysis/photos/13.png
processing /home/meso/data/cross-section-photos/analysis/photos/14.png
proces